# Airline Customer Satisfaction Capstone
## EDA
## Cleaning The Dataset

#### What problems do an airline face?

1: There are 5000 airlines currently operating. This means relative to other industries there has been little consolidation so competition for passangers is fierce.
2: Flyers have significant choice on which airline to fly with.
3: COVID restrictions and labour shortages have made operating airlines even more challenging.

#### Why do airlines need to know about customer satisfaction?

1: There is limited capital to invest.
2: Airlines want to know what to invest in with regards to retaining existing customers whilst attracting new ones.
3: Airlines want to know which customers to focus on.


#### What does this project aim to do?

Whether it's deciding on what refreshments to offer or how much to invest in the online booking process, this project mimicks the type of analysis a data scientist would conduct to answer these types of questions.

#### Where is the data from?

This data is sourced from John.D on Kaggle: https://www.kaggle.com/datasets/johndddddd/customer-satisfaction.

The data is reportadley from an airline survery, though the exact source is not shared. 

This means the data suffers the general disadvantages of survery data:

1: Valuable data is missing from individuals who did not fill in the survery which could contradict our findings.

2: The data provided is what individuals believe their satisfaction to be is, but this may not represent their true satisfaction.

In [1]:
# Below I am importing the relavent libraries and packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import random
np.random.seed(123)
random.seed(123)

from sklearn.model_selection import train_test_split

# Filter warnings
import warnings
warnings.filterwarnings('ignore')

import os
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.compose import ColumnTransformer

In [3]:
# Below I am setting df to our data
os.getcwd()
df = pd.read_excel("satisfaction_2015.xlsx")

In [4]:
# Below I am looking at the first 2 rows of the dataframe
df.head(2)

,id,satisfaction_v2,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,...,Seat comfort,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes
0,117135,satisfied,Male,disloyal Customer,56,Personal Travel,Eco,369,0,2,...,3,3,1,5,3,3,4,3,0,0.0
1,72091,satisfied,Male,disloyal Customer,49,Personal Travel,Eco,2486,0,2,...,3,2,1,1,4,4,3,2,0,0.0


In [5]:
# Below I am looking at the shape of the dataframe:
df.shape

(129880, 24)

### Removing Missing/Null Values

In [6]:
# Below i am identifying if any columns have any null values
df_nulls = df.isna().any()
df_nulls.value_counts()

False    23
True      1
dtype: int64

In [7]:
# Below I am identifying the % of null values in each column
df.isnull().sum()/len(df)

id                                   0.000000
satisfaction_v2                      0.000000
Gender                               0.000000
Customer Type                        0.000000
Age                                  0.000000
Type of Travel                       0.000000
Class                                0.000000
Flight Distance                      0.000000
Inflight wifi service                0.000000
Departure/Arrival time convenient    0.000000
Ease of Online booking               0.000000
Gate location                        0.000000
Food and drink                       0.000000
Online boarding                      0.000000
Seat comfort                         0.000000
Inflight entertainment               0.000000
On-board service                     0.000000
Leg room service                     0.000000
Baggage handling                     0.000000
Checkin service                      0.000000
Inflight service                     0.000000
Cleanliness                       

In [8]:
# Below I am identifying the rows which have null values
df[df['Arrival Delay in Minutes'].isnull()]

,id,satisfaction_v2,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,...,Seat comfort,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes
194,44820,satisfied,Male,disloyal Customer,49,Personal Travel,Eco,491,5,4,...,3,3,1,4,3,3,4,3,0,NaN
197,46936,satisfied,Female,disloyal Customer,63,Personal Travel,Eco,776,5,5,...,3,3,5,2,5,3,5,3,7,NaN
351,61021,satisfied,Female,Loyal Customer,21,Personal Travel,Eco,3365,0,2,...,5,5,4,1,4,2,3,5,20,NaN
452,40480,satisfied,Male,Loyal Customer,28,Personal Travel,Eco,222,0,3,...,4,2,1,1,3,4,3,2,0,NaN
916,19597,satisfied,Male,Loyal Customer,28,Personal Travel,Eco,173,0,5,...,5,5,4,4,5,3,4,5,24,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127457,125075,satisfied,Female,Loyal Customer,41,Business travel,Business,3758,1,1,...,5,5,5,5,5,4,5,5,23,NaN
127658,75644,satisfied,Female,Loyal Customer,60,Business travel,Business,3041,3,3,...,4,5,5,5,5,4,5,3,0,NaN
127945,40434,satisfied,Male,Loyal Customer,44,Business travel,Business,3383,2,2,...,4,5,5,5,5,5,5,4,70,NaN
128337,120101,satisfied,Male,Loyal Customer,39,Business travel,Business,1576,4,4,...,5,5,5,5,5,3,5,5,0,NaN


If the missing values where something we could fix (like lattitude, longitude etc.) then we could try to replace the missing values with correct ones. Here this is not the case so our best course of action is to delete these rows. Luckily they account for a small % of the overall dataset.

In [9]:
# Below I am dropping the null values in the 'Arrival Delay in Minutes' column
df = df.dropna(how='any',axis=0,thresh=None, subset=None, inplace=False)

In [10]:
# Below I am sanity checking that the null values have been removed
df.isnull().sum()/len(df)

id                                   0.0
satisfaction_v2                      0.0
Gender                               0.0
Customer Type                        0.0
Age                                  0.0
Type of Travel                       0.0
Class                                0.0
Flight Distance                      0.0
Inflight wifi service                0.0
Departure/Arrival time convenient    0.0
Ease of Online booking               0.0
Gate location                        0.0
Food and drink                       0.0
Online boarding                      0.0
Seat comfort                         0.0
Inflight entertainment               0.0
On-board service                     0.0
Leg room service                     0.0
Baggage handling                     0.0
Checkin service                      0.0
Inflight service                     0.0
Cleanliness                          0.0
Departure Delay in Minutes           0.0
Arrival Delay in Minutes             0.0
dtype: float64

Below I am checking for any rows with null values

In [29]:
def nans(df): return df[df.isnull().any(axis=1)]

In [34]:
print(f"There are {row_null.shape[0]} rows with missing values.")

There are 0 rows with missing values.


### Removing Duplicated Values

In [11]:
# Below I am checking for any duplicated rows
df.duplicated().sum()

0

### Analysing Column Data Types

We need to understand what each column is showing. We can look at the datatype of each column:

In [21]:
# Below I am looking at the using the built in 'info()' function to identify the datatype of each column
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 129487 entries, 0 to 129879
Data columns (total 23 columns):
 #   Column                             Non-Null Count   Dtype 
---  ------                             --------------   ----- 
 0   satisfaction_v2                    129487 non-null  object
 1   Gender                             129487 non-null  object
 2   Customer Type                      129487 non-null  object
 3   Age                                129487 non-null  int64 
 4   Type of Travel                     129487 non-null  object
 5   Class                              129487 non-null  object
 6   Flight Distance                    129487 non-null  int64 
 7   Inflight wifi service              129487 non-null  int64 
 8   Departure/Arrival time convenient  129487 non-null  int64 
 9   Ease of Online booking             129487 non-null  int64 
 10  Gate location                      129487 non-null  int64 
 11  Food and drink                     129487 non-null  

The id of a customer has no relavence to if a customer is satisfied or not, since the id is assigned at random. Therefore we can remove it from our dataframe.

In [19]:
# Below I am checking the sum of 'Arrival Delay in Minutes'
df['Arrival Delay in Minutes'].sum()

1954105

We can see that the sum of 'Arrival Delay in Minutes' is a whole number, which means it can be stored as an int data type.

In [20]:
# Below I am changing the the 'Arrival Delay in Minutes' column to the int datatype
df['Arrival Delay in Minutes'] = df['Arrival Delay in Minutes'].astype(int)

### Renaming and Dropping Columns

First I need to get a better understanding of what each column represents.

In [22]:
print("-------------------------------------------------------------------")
print(f"The unique values for 'satisfaction_v2' are:")
print("\n")
print(df['satisfaction_v2'].value_counts())
print("-------------------------------------------------------------------")
print(f"The unique values for 'Gender' are:")
print("\n")
print(df['Gender'].value_counts())
print("-------------------------------------------------------------------")
print(f"The unique values for 'Customer Type 'are:")
print("\n")
print(df['Customer Type'].value_counts())
print("-------------------------------------------------------------------")
print(f"The unique values for 'Age 'are:")
print("\n")
print(df['Age'].value_counts())
print("-------------------------------------------------------------------")
print(f"The unique values for 'Type of Travel'are:")
print("\n")
print(df['Type of Travel'].value_counts())
print("-------------------------------------------------------------------")
print(f"The unique values for 'Class'are:")
print("\n")
print(df['Class'].value_counts())
print("-------------------------------------------------------------------")
print(f"The unique values for 'Flight Distance'are:")
print("\n")
print(df['Flight Distance'].value_counts())
print("-------------------------------------------------------------------")
print(f"The unique values for 'Inflight wifi' service'are:")
print("\n")
print(df['Inflight wifi service'].value_counts())
print("-------------------------------------------------------------------")
print(f"The unique values for 'Departure/Arrival' time convenient'are:")
print("\n")
print(df['Departure/Arrival time convenient'].value_counts())
print("-------------------------------------------------------------------")
print(f"The unique values for 'Ease of Online booking' time convenient'are:")
print("\n")
print(df['Ease of Online booking'].value_counts())
print("-------------------------------------------------------------------")
print(f"The unique values for 'Gate location' time convenient'are:")
print("\n")
print(df['Gate location'].value_counts())
print("-------------------------------------------------------------------")
print(f"The unique values for 'Food and drink' time convenient'are:")
print("\n")
print(df['Food and drink'].value_counts())
print("-------------------------------------------------------------------")
print(f"The unique values for 'Online boarding' time convenient'are:")
print("\n")
print(df['Online boarding'].value_counts())
print("-------------------------------------------------------------------")
print(f"The unique values for 'Seat comfort' time convenient'are:")
print("\n")
print(df['Seat comfort'].value_counts())
print("-------------------------------------------------------------------")
print(f"The unique values for 'Inflight entertainment' time convenient'are:")
print("\n")
print(df['Inflight entertainment'].value_counts())
print("-------------------------------------------------------------------")
print(f"The unique values for 'On-board service' time convenient'are:")
print("\n")
print(df['On-board service'].value_counts())
print("-------------------------------------------------------------------")
print(f"The unique values for 'Leg room service' time convenient'are:")
print("\n")
print(df['Leg room service'].value_counts())
print("-------------------------------------------------------------------")
print(f"The unique values for 'Baggage handling' time convenient'are:")
print("\n")
print(df['Baggage handling'].value_counts())
print("-------------------------------------------------------------------")
print(f"The unique values for 'Checkin service' time convenient'are:")
print("\n")
print(df['Checkin service'].value_counts())
print("-------------------------------------------------------------------")
print(f"The unique values for 'Inflight service' time convenient'are:")
print("\n")
print(df['Inflight service'].value_counts())
print("-------------------------------------------------------------------")
print(f"The unique values for 'Cleanliness' time convenient'are:")
print("\n")
print(df['Cleanliness'].value_counts())
print("-------------------------------------------------------------------")
print(f"The unique values for 'Departure Delay in Minutes' time convenient'are:")
print("\n")
print(df['Departure Delay in Minutes'].value_counts())
print("-------------------------------------------------------------------")
print(f"The unique values for 'Arrival Delay in Minutes' time convenient'are:")
print("\n")
print(df['Arrival Delay in Minutes'].value_counts())
print("-------------------------------------------------------------------")

-------------------------------------------------------------------
The unique values for 'satisfaction_v2' are:


neutral or dissatisfied    73225
satisfied                  56262
Name: satisfaction_v2, dtype: int64
-------------------------------------------------------------------
The unique values for 'Gender' are:


Female    65703
Male      63784
Name: Gender, dtype: int64
-------------------------------------------------------------------
The unique values for 'Customer Type 'are:


Loyal Customer       105773
disloyal Customer     23714
Name: Customer Type, dtype: int64
-------------------------------------------------------------------
The unique values for 'Age 'are:


39    3681
25    3501
40    3203
44    3099
41    3081
      ... 
74      61
76      60
79      52
78      44
85      25
Name: Age, Length: 75, dtype: int64
-------------------------------------------------------------------
The unique values for 'Type of Travel'are:


Business travel    89445
Personal Travel  

#### Data Description v1

Categorical Columns:
- `satisfaction_v2` --> This is our target feature. It is whether or not a customer was satisfied
- `Gender` --> This shows whether or not the customer was male or female
- `Type of Travel` --> This shows whether or not the customer travelled for business or personal travel
- `Class` --> This is the type of class the customer flew in (Eco, Eco Plus, Business)
- `Gate location` --> This is the satisfaction rating of gate location

Numerical Column With No Range Limit:
- `Age` --> This is the age of the customer
- `Flight Distance` --> This is the flight distance in miles
- `Gate location` --> This is the satisfaction rating of gate location
- `Departure Delay in Minutes` --> This is the numnber of minutes the flight was delayed at departure
- `Arrival Delay in Minutes` --> This is the numnber of minutes the flight was delayed at arrival to destination

Numerical Column Rating 0 - 5:
- `Inflight wifi` --> This is the satisfaction rating of the inflight wifi
- `Departure/Arrival time` --> This is the satisfaction rating of the inflight wifi
- `Ease of Online booking` --> This is the satisfaction rating of the online booking process
- `Food and drink` --> This is the satisfaction rating of the food and drink offered in flight
- `Online boarding` --> This is the satisfaction rating of the online boarding
- `Seat comfort` --> This is the satisfaction rating of the inflight seats
- `Inflight entertainment` --> This is the satisfaction rating of the inflight entertainment
- `On-board service` --> This is the satisfaction rating of onboard service
- `Baggage handling` --> This is the satisfaction rating of baggage handling
- `Checkin service` --> This is the satisfaction rating of the checkin service
- `Leg room service` --> This is the satisfaction rating of inflight leg room
- `Inflight service` --> This is the satisfaction rating of the inflight service
- `Cleanliness` --> This is the satisfaction rating of how clean the aeroplane

These column names need to be optimised so they best describe what they are showing.

In [23]:
df.rename(columns = {'satisfaction_v2' : 'satisfaction_target'}, inplace=True)
df.rename(columns = {'Gender': 'customer_gender'}, inplace=True)
df.rename(columns = { 'Customer Type' : 'customer_loyalty_type'}, inplace=True)
df.rename(columns = {'Age': 'customer_age'}, inplace=True)
df.rename(columns = {'Type of Travel' : 'customer_travel_type'}, inplace=True)
df.rename(columns = {'Class' : 'customer_class_type'}, inplace=True)
df.rename(columns = {'Flight Distance' : 'flight_distance'}, inplace=True)
df.rename(columns = {'Inflight wifi service' : 'flight_wifi_satisfaction_rating'}, inplace=True)
df.rename(columns = {'Departure/Arrival time convenient' : 'departure/arrival_time_satisfaction_rating'}, inplace=True)
df.rename(columns = {'Ease of Online booking' : 'online_booking_satisfaction_rating'}, inplace=True)
df.rename(columns = {'Gate location' : 'gate_location_satisfaction_rating'}, inplace=True)
df.rename(columns = {'Food and drink' : 'food/drink_satisfaction_rating'}, inplace=True)
df.rename(columns = {'Online boarding' : 'online_boarding_satisfaction_rating'}, inplace=True)
df.rename(columns = {'Seat comfort' : 'seat_comfort_satisfaction_rating'}, inplace=True)
df.rename(columns = {'Inflight entertainment' : 'inflight_entertainment_satisfaction_rating'}, inplace=True)
df.rename(columns = {'On-board service' : 'onboard_service_satisfaction_rating'}, inplace=True)
df.rename(columns = {'Leg room service' : 'leg_room_n_satisfaction_rating'}, inplace=True)
df.rename(columns = {'Baggage handling' : 'baggage_handling_satisfaction_rating'}, inplace=True)
df.rename(columns = {'Checkin service' : 'checkin_service_satisfaction_rating'}, inplace=True)
df.rename(columns = {'Inflight service' : 'inflight_service_satisfaction_rating'}, inplace=True)
df.rename(columns = {'Cleanliness' : 'cleanliness_satisfaction_rating'}, inplace=True)
df.rename(columns = {'Departure Delay in Minutes' : 'departure_delay_in_minutes'}, inplace=True)
df.rename(columns = {'Arrival Delay in Minutes' : 'arrival_delay_in_minutes'}, inplace=True)

In [25]:
# Below I am sanity checking that the column names have been changed correctly
df.head(2)

,satisfaction_target,customer_gender,customer_loyalty_type,customer_age,customer_travel_type,customer_class_type,flight_distance,flight_wifi_satisfaction_rating,departure/arrival_time_satisfaction_rating,online_booking_satisfaction_rating,...,seat_comfort_satisfaction_rating,inflight_entertainment_satisfaction_rating,onboard_service_satisfaction_rating,leg_room_n_satisfaction_rating,baggage_handling_satisfaction_rating,checkin_service_satisfaction_rating,inflight_service_satisfaction_rating,cleanliness_satisfaction_rating,departure_delay_in_minutes,arrival_delay_in_minutes
0,satisfied,Male,disloyal Customer,56,Personal Travel,Eco,369,0,2,0,...,3,3,1,5,3,3,4,3,0,0
1,satisfied,Male,disloyal Customer,49,Personal Travel,Eco,2486,0,2,1,...,3,2,1,1,4,4,3,2,0,0


In [36]:
df.to_csv('cleaned_airlines.csv')